# Block Aligner Benchmark Analysis and Visualizations

This notebook contains code for collecting, cleaning, and analyzing data produced by block aligner's experiments.

To run this, you will need to install all the libraries imported below, along with [altair-saver](https://github.com/altair-viz/altair_saver) and [altair-data-server](https://github.com/altair-viz/altair_data_server), which has some extra dependencies for PDF saving.

Run each cell one by one to reproduce the experiments. This may take a while. For accurate benchmarking, it is recommended to run the entire notebook in the command line with `nbconvert`.

In [1]:
import altair as alt
from altair_saver import save
from altair import datum
import pandas as pd
from io import StringIO

alt.data_transformers.enable("data_server")

DataTransformerRegistry.enable('data_server')

In [2]:
def csv_to_pandas(csv, d = "\\s*,\\s*", t = None):
    s = StringIO("\n".join(csv))
    data = pd.read_csv(s, sep = d, thousands = t, comment = "#", engine = "python")
    return data

## Prefix Scan Benchmark

In [3]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- prefix_scan | grep 'bench:' | awk '{print $2"\t"$5}'
output.insert(0, "algorithm\ttime")
output

['algorithm\ttime', 'bench_naive_prefix_scan\t34', 'bench_opt_prefix_scan\t17']

In [4]:
data = csv_to_pandas(output, d = "\t", t = ",")
data

,algorithm,time
0,bench_naive_prefix_scan,34
1,bench_opt_prefix_scan,17


In [5]:
data["algorithm"] = data["algorithm"].map({
    "bench_naive_prefix_scan": "naive",
    "bench_opt_prefix_scan": "ours"
})
data

,algorithm,time
0,naive,34
1,ours,17


Prefix Scan Benchmark (AVX2)

In [6]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("time", axis = alt.Axis(title = "time (ns)")),
    y = "algorithm",
    color = alt.Color("algorithm", legend = None)
).properties(
    width = 150
)
save(c, "prefix_scan_bench.pdf")
c

alt.Chart(...)

## Random Data Benchmark

In [7]:
output = !cd .. && cargo bench --features simd_avx2 --quiet -- bench_ | grep 'bench:' | grep -v 'prefix_scan' | awk '{print $2"\t"$5}'
output

['bench_parasailors_aa_1000_10000\t48,155,857',
 'bench_parasailors_aa_100_1000\t504,507',
 'bench_parasailors_aa_10_100\t18,084',
 'bench_rustbio_aa_100_1000\t14,432,734',
 'bench_rustbio_aa_10_100\t158,834',
 'bench_scan_aa_1000_10000\t232,365',
 'bench_scan_aa_1000_10000_insert\t8,819,136',
 'bench_scan_aa_1000_10000_small\t215,387',
 'bench_scan_aa_1000_10000_trace\t1,884,197',
 'bench_scan_aa_100_1000\t23,840',
 'bench_scan_aa_100_1000_insert\t50,314',
 'bench_scan_aa_100_1000_small\t23,744',
 'bench_scan_aa_100_1000_trace\t517,001',
 'bench_scan_aa_10_100\t3,881',
 'bench_scan_aa_10_100_insert\t4,054',
 'bench_scan_aa_10_100_small\t3,544',
 'bench_scan_aa_10_100_trace\t356,228',
 'bench_scan_nuc_1000_10000\t231,542',
 'bench_scan_nuc_100_1000\t23,299',
 'bench_triple_accel_1000_10000\t8,386,151',
 'bench_triple_accel_100_1000\t25,679']

In [8]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["parasailors_aa", "rustbio_aa", "scan_aa", "scan_nuc", "triple_accel"]
new_names = ["parasailors\tprotein", "rust bio\tprotein", "ours\tprotein", "ours\tnucleotide", "triple accel\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'parasailors\tprotein\t1000\t10000\tdefault\t48,155,857',
 'parasailors\tprotein\t100\t1000\tdefault\t504,507',
 'parasailors\tprotein\t10\t100\tdefault\t18,084',
 'rust bio\tprotein\t100\t1000\tdefault\t14,432,734',
 'rust bio\tprotein\t10\t100\tdefault\t158,834',
 'ours\tprotein\t1000\t10000\tdefault\t232,365',
 'ours\tprotein\t1000\t10000\tinsert\t8,819,136',
 'ours\tprotein\t1000\t10000\tsmall\t215,387',
 'ours\tprotein\t1000\t10000\ttrace\t1,884,197',
 'ours\tprotein\t100\t1000\tdefault\t23,840',
 'ours\tprotein\t100\t1000\tinsert\t50,314',
 'ours\tprotein\t100\t1000\tsmall\t23,744',
 'ours\tprotein\t100\t1000\ttrace\t517,001',
 'ours\tprotein\t10\t100\tdefault\t3,881',
 'ours\tprotein\t10\t100\tinsert\t4,054',
 'ours\tprotein\t10\t100\tsmall\t3,544',
 'ours\tprotein\t10\t100\ttrace\t356,228',
 'ours\tnucleotide\t1000\t10000\tdefault\t231,542',
 'ours\tnucleotide\t100\t1000\tdefault\t23,299',
 'triple accel\tnucleotide\t1000\t100

In [9]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,parasailors,protein,1000,10000,default,48155857
1,parasailors,protein,100,1000,default,504507
2,parasailors,protein,10,100,default,18084
3,rust bio,protein,100,1000,default,14432734
4,rust bio,protein,10,100,default,158834
5,ours,protein,1000,10000,default,232365
6,ours,protein,1000,10000,insert,8819136
7,ours,protein,1000,10000,small,215387
8,ours,protein,1000,10000,trace,1884197
9,ours,protein,100,1000,default,23840


In [10]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (AVX2)

In [11]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench.pdf")
c

alt.Chart(...)

Random DNA Sequences Benchmark (AVX2)

In [12]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log", domain = [1, 50000])),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000])),
    shape = alt.Color("length:N", scale = alt.Scale(domain = [100, 1000, 10000]))
).transform_filter(
    datum.alphabet == "nucleotide"
).properties(
    width = 200,
    height = 50
)
save(c, "random_dna_bench.pdf")
c

alt.Chart(...)

## Uniclust 30 Data Benchmark

In [13]:
output = !cd .. && cargo run --example uc_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, size, time',
 'ours (no trace), uc30, 32-32, 0.056172268',
 'ours (no trace), uc30 0.95, 32-32, 0.060529935',
 'ours (no trace), uc30, 32-256, 0.088333406',
 'ours (no trace), uc30 0.95, 32-256, 0.076304708',
 'ours (no trace), uc30, 256-256, 0.198884566',
 'ours (no trace), uc30 0.95, 256-256, 0.216352655',
 'ours (trace), uc30, 32-256, 0.184920397',
 'ours (trace), uc30 0.95, 32-256, 0.160019013',
 'parasail, uc30, full, 0.81885358',
 'parasail, uc30 0.95, full, 0.952500321']

In [14]:
data = csv_to_pandas(output)
data

,algorithm,dataset,size,time
0,ours (no trace),uc30,32-32,0.056172
1,ours (no trace),uc30 0.95,32-32,0.060530
2,ours (no trace),uc30,32-256,0.088333
3,ours (no trace),uc30 0.95,32-256,0.076305
4,ours (no trace),uc30,256-256,0.198885
5,ours (no trace),uc30 0.95,256-256,0.216353
6,ours (trace),uc30,32-256,0.184920
7,ours (trace),uc30 0.95,32-256,0.160019
8,parasail,uc30,full,0.818854
9,parasail,uc30 0.95,full,0.952500


Uniclust30 Benchmark (AVX2)

In [15]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("algorithm", axis = None),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = "algorithm"
).transform_filter(
    (datum.size == "32-256") | (datum.algorithm == "parasail")
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
).configure_range(
    category = {"scheme": "dark2"}
)
save(c, "uniclust30_bench.pdf")
c

alt.FacetChart(...)

Uniclust30 Block Size Benchmark (AVX2)

In [16]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", axis = None, sort = ["32-32", "32-256", "256-256"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)"), scale = alt.Scale(domain = [0.0, 1.0])),
    color = alt.Color("size", sort = ["32-32", "32-256", "256-256"])
).transform_filter(
    datum.algorithm == "ours (no trace)"
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = (c + t).properties(
    width = 50,
    height = 100
).facet(
    column = alt.Column("dataset", header = alt.Header(orient = "bottom")),
)
save(c, "uniclust30_size_bench.pdf")
c

alt.FacetChart(...)

## Nanopore Data Benchmark Setup

To run the benchmarks below, you need to clone the following repos, place them in the same directory where this repo (block aligner) is located, and follow their setup instructions:
* [diff-bench-paper](https://github.com/Daniel-Liu-c0deb0t/diff-bench-paper)
* [adaptivebandbench](https://github.com/Daniel-Liu-c0deb0t/adaptivebandbench)

## Nanopore Data Benchmark

In [17]:
output = !cd .. && cargo run --example nanopore_bench --release --features simd_avx2 --quiet
output

['# time (s)',
 'algorithm, dataset, time',
 'ours (no trace 32-32), nanopore 25kbp, 0.945440575',
 'ours (no trace 32-32), random, 2.334440348',
 'ours (trace 32-32), nanopore 25kbp, 1.3234968710000001',
 'ours (trace 32-32), random, 3.236023942',
 'ours (trace 32-64), nanopore 25kbp, 1.669813312',
 'ours (trace 32-64), random, 3.545986289']

In [18]:
data = csv_to_pandas(output)
data

,algorithm,dataset,time
0,ours (no trace 32-32),nanopore 25kbp,0.945441
1,ours (no trace 32-32),random,2.334440
2,ours (trace 32-32),nanopore 25kbp,1.323497
3,ours (trace 32-32),random,3.236024
4,ours (trace 32-64),nanopore 25kbp,1.669813
5,ours (trace 32-64),random,3.545986


In [19]:
output2 = !cd ../../diff-bench-paper/supplementary_data/benchmark_codes && ./custom_bench.sh

for i, o in enumerate(output2):
    if o.startswith("cells("):
        break
output2 = output2[i + 1:]

output2.insert(0, "algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail")
output2

['algorithm\tfill time\ttrace time\tconvert time\ttotal time\tscore\tfail',
 'editdist\t467377000\t168255000\t66259000\t701891000\t6880489\t0',
 'non-diff\t665058000\t273896000\t59740000\t998694000\t27124786\t52',
 'diff-raw\t618038000\t215400000\t62934000\t896372000\t27291141\t32',
 'libgaba\t435779000\t150753000\t31633000\t618165000\t27121546\t53',
 'edlib\t28112512000\t19433150000\t107151000\t47652813000\t37\t0',
 'seqan\t90359835000\t0\t0\t90359835000\t0\t0']

In [20]:
data2 = csv_to_pandas(output2, d = "\t")
data2

,algorithm,fill time,trace time,convert time,total time,score,fail
0,editdist,467377000,168255000,66259000,701891000,6880489,0
1,non-diff,665058000,273896000,59740000,998694000,27124786,52
2,diff-raw,618038000,215400000,62934000,896372000,27291141,32
3,libgaba,435779000,150753000,31633000,618165000,27121546,53
4,edlib,28112512000,19433150000,107151000,47652813000,37,0
5,seqan,90359835000,0,0,90359835000,0,0


In [21]:
cleaned2 = data2.drop(columns = ["trace time", "convert time", "total time", "score", "fail"])
cleaned2 = cleaned2.rename(columns = {"fill time": "time"})
cleaned2["time"] /= 1e9
cleaned2

,algorithm,time
0,editdist,0.467377
1,non-diff,0.665058
2,diff-raw,0.618038
3,libgaba,0.435779
4,edlib,28.112512
5,seqan,90.359835


In [22]:
cleaned = data.drop(index = [1, 3, 5])
cleaned = cleaned.drop(columns = ["dataset"])
cleaned = cleaned.append(cleaned2, ignore_index = True)
cleaned

,algorithm,time
0,ours (no trace 32-32),0.945441
1,ours (trace 32-32),1.323497
2,ours (trace 32-64),1.669813
3,editdist,0.467377
4,non-diff,0.665058
5,diff-raw,0.618038
6,libgaba,0.435779
7,edlib,28.112512
8,seqan,90.359835


25kbp Nanopore Reads Benchmark (AVX2)

In [23]:
chart1 = alt.Chart(cleaned).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm != "ours (trace 32-32)") & (datum.algorithm != "ours (no trace 32-32)") & (datum.algorithm != "ours (trace 32-64)"))

chart2 = alt.Chart(cleaned).mark_point(color = "red", filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (s)", grid = True), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm", axis = alt.Axis(grid = True), sort = alt.EncodingSortField(field = "time"))
).transform_filter((datum.algorithm == "ours (trace 32-32)") | (datum.algorithm == "ours (no trace 32-32)") | (datum.algorithm == "ours (trace 32-64)"))

c = (chart1 + chart2).properties(
    width = 150,
    height = 150
)
save(c, "nanopore_bench.pdf")
c

alt.LayerChart(...)

## Sequence-to-Profile Alignment Benchmark

In [24]:
output = !cd .. && cargo run --example pssm_bench --release --features simd_avx2 --quiet
output

['size, time',
 '32-32, 0.155743695',
 '32-64, 0.181100085',
 '32-128, 0.214255002',
 '128-128, 0.216553961',
 '# Done!']

In [25]:
data = csv_to_pandas(output)
data

,size,time
0,32-32,0.155744
1,32-64,0.181100
2,32-128,0.214255
3,128-128,0.216554


SCOP Sequence-to-Profile Alignment Benchmark (AVX2)

In [26]:
c = alt.Chart(data).mark_bar().encode(
    x = alt.X("size", sort = ["32-32", "32-64", "32-128", "128-128"]),
    y = alt.Y("time", axis = alt.Axis(title = "time (s)")),
    color = alt.Color("size", sort = ["32-32", "32-64", "32-128", "128-128"], legend = None)
).transform_filter(
    datum.size != "2048-2048"
).properties(
    width = 60,
    height = 100
)
t = c.mark_text(dy = -4, size = 7).encode(text = alt.Text("time", format = ".2"))
c = c + t
save(c, "pssm_size_bench.pdf")
c

alt.LayerChart(...)

## WASM SIMD

[Wasmtime](https://wasmtime.dev/) is needed to run the webassembly code.

In [27]:
output = !CARGO_TARGET_WASM32_WASI_RUNNER="wasmtime --wasm-features simd --" cargo bench --target=wasm32-wasi --features simd_wasm --quiet -- --nocapture | grep 'bench:' | awk '{print $2"\t"$5}'
output

['bench_rustbio_aa_100_1000\t24,199,903',
 'bench_rustbio_aa_10_100\t249,137',
 'bench_scan_aa_1000_10000\t1,029,175',
 'bench_scan_aa_1000_10000_insert\t11,719,417',
 'bench_scan_aa_1000_10000_small\t672,887',
 'bench_scan_aa_1000_10000_trace\t2,708,199',
 'bench_scan_aa_100_1000\t94,713',
 'bench_scan_aa_100_1000_insert\t181,724',
 'bench_scan_aa_100_1000_small\t67,035',
 'bench_scan_aa_100_1000_trace\t395,282',
 'bench_scan_aa_10_100\t8,390',
 'bench_scan_aa_10_100_insert\t11,960',
 'bench_scan_aa_10_100_small\t5,999',
 'bench_scan_aa_10_100_trace\t192,699',
 'bench_scan_nuc_1000_10000\t673,740',
 'bench_scan_nuc_100_1000\t67,624']

In [28]:
cleaned = ["algorithm\talphabet\tk\tlength\tproperty\ttime"]
names = ["rustbio_aa", "scan_aa", "scan_nuc"]
new_names = ["rust bio\tprotein", "ours\tprotein", "ours\tnucleotide"]

for o in output:
    o = o[len("bench_"):]
    for n, nn in zip(names, new_names):
        if o.startswith(n):
            suffix = o[len(n):].replace("_", "\t")
            o = nn + suffix
            break
    if len(o.split("\t")) < len(cleaned[0].split("\t")):
        insert_idx = o.rindex("\t")
        o = o[:insert_idx] + "\tdefault" + o[insert_idx:]
    cleaned.append(o)

cleaned

['algorithm\talphabet\tk\tlength\tproperty\ttime',
 'rust bio\tprotein\t100\t1000\tdefault\t24,199,903',
 'rust bio\tprotein\t10\t100\tdefault\t249,137',
 'ours\tprotein\t1000\t10000\tdefault\t1,029,175',
 'ours\tprotein\t1000\t10000\tinsert\t11,719,417',
 'ours\tprotein\t1000\t10000\tsmall\t672,887',
 'ours\tprotein\t1000\t10000\ttrace\t2,708,199',
 'ours\tprotein\t100\t1000\tdefault\t94,713',
 'ours\tprotein\t100\t1000\tinsert\t181,724',
 'ours\tprotein\t100\t1000\tsmall\t67,035',
 'ours\tprotein\t100\t1000\ttrace\t395,282',
 'ours\tprotein\t10\t100\tdefault\t8,390',
 'ours\tprotein\t10\t100\tinsert\t11,960',
 'ours\tprotein\t10\t100\tsmall\t5,999',
 'ours\tprotein\t10\t100\ttrace\t192,699',
 'ours\tnucleotide\t1000\t10000\tdefault\t673,740',
 'ours\tnucleotide\t100\t1000\tdefault\t67,624']

In [29]:
data = csv_to_pandas(cleaned, d = "\t", t = ",")
data

,algorithm,alphabet,k,length,property,time
0,rust bio,protein,100,1000,default,24199903
1,rust bio,protein,10,100,default,249137
2,ours,protein,1000,10000,default,1029175
3,ours,protein,1000,10000,insert,11719417
4,ours,protein,1000,10000,small,672887
5,ours,protein,1000,10000,trace,2708199
6,ours,protein,100,1000,default,94713
7,ours,protein,100,1000,insert,181724
8,ours,protein,100,1000,small,67035
9,ours,protein,100,1000,trace,395282


In [30]:
data["algorithm property"] = data["algorithm"] + " " + data["property"]
data["time"] /= 1000

Random Protein Sequences Benchmark (WASM SIMD)

In [31]:
c = alt.Chart(data).mark_point(opacity = 1, filled = True).encode(
    x = alt.X("time", axis = alt.Axis(title = "time (us)"), scale = alt.Scale(type = "log")),
    y = alt.Y("algorithm property", axis = alt.Axis(title = "algorithm", grid = True), sort = alt.EncodingSortField(field = "time")),
    color = "length:N",
    shape = "length:N"
).transform_filter(
    datum.alphabet == "protein"
).properties(
    width = 200,
    height = 150
)
save(c, "random_protein_bench_wasm.pdf")
c

alt.Chart(...)